# 1. Contextualização dos Dados para Agrupamento

Para o modelo de Clusterização (Não-Supervisionado), o objetivo é segmentar usuários para identificar vulnerabilidade social.

**Geração dos Dados Demográficos:**
Os dados de `renda_familiar`, `tempo_desemprego` e `idade` foram gerados de forma aleatória dentro de intervalos realistas (ex: renda entre R$ 1.500 e R$ 15.000), mas sem rótulos prévios de "classe social".

O objetivo deste notebook é utilizar o algoritmo K-Means para descobrir essas classes (clusters) naturalmente, baseando-se na proximidade desses valores numéricos.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import joblib

# --- 1. CARREGAMENTO ---
df = pd.read_csv('dados_recomece.csv')

# Seleção de features para Agrupamento Social
features_cluster = ['idade', 'tempo_desemprego', 'renda_familiar']
dados_selecionados = df[features_cluster]

# --- 2. ANÁLISE EXPLORATÓRIA (EDA) ---
print(">>> Analisando distribuição das variáveis de cluster...")
plt.figure(figsize=(15, 5))
for i, col in enumerate(features_cluster):
    plt.subplot(1, 3, i+1)
    sns.histplot(df[col], kde=True, color='skyblue')
    plt.title(f'Distribuição: {col}')
plt.tight_layout()
plt.show()

# --- 3. PRÉ-PROCESSAMENTO (ESCALONAMENTO) ---
# Agrupamento é sensível à escala (Renda é muito maior que Idade), então precisamos normalizar
scaler = StandardScaler()
dados_scaled = scaler.fit_transform(dados_selecionados)

# --- 4. MÉTODO DO COTOVELO (Definindo K) ---
# Vamos testar de 1 a 10 clusters para ver o número ideal
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42, n_init=10)
    kmeans.fit(dados_scaled)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(range(1, 11), wcss, marker='o', linestyle='--')
plt.title('Método do Cotovelo (Elbow Method)')
plt.xlabel('Número de Clusters')
plt.ylabel('WCSS (Soma dos quadrados intra-cluster)')
plt.show()
print(">>> Visualmente, o 'cotovelo' indica que 3 grupos é um bom número.")

# --- 5. TREINAMENTO DO MODELO FINAL ---
kmeans_final = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans_final.fit(dados_scaled)

# --- 6. ANÁLISE DOS RESULTADOS ---
# Adicionar o cluster de volta aos dados originais para interpretar
df['cluster_social'] = kmeans_final.labels_

print("\n>>> Perfil Médio de cada Cluster:")
perfil_grupos = df.groupby('cluster_social')[features_cluster].mean()
display(perfil_grupos)

# Visualização dos Clusters (Idade x Renda)
plt.figure(figsize=(10, 6))
sns.scatterplot(x='idade', y='renda_familiar', hue='cluster_social', data=df, palette='viridis', s=100)
plt.title('Segmentação de Usuários (Clusters)')
plt.show()

# --- 7. SALVAMENTO ---
joblib.dump(kmeans_final, 'modelo_cluster.joblib')
joblib.dump(scaler, 'scaler_cluster.joblib') # Importante salvar o scaler também!
print("\n>>> Modelos salvos: 'modelo_cluster.joblib' e 'scaler_cluster.joblib'")

              renda_familiar  tempo_desemprego
grupo_social                                  
0                8491.758065         11.430108
1                6874.732308         18.590769
2                9305.244224          6.056106
Modelos de cluster salvos.
